In [15]:
import pandas as pd
import math
import numpy as np
import datetime 

In [16]:
dataset = pd.read_csv('./datasets/dataset_validation.csv')


dataset = dataset.drop(columns=['Unnamed: 0'])
dataset = dataset.rename(columns={"block_time":"timestamp"})
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])
dataset['timestamp'] = dataset['timestamp'].dt.strftime('%Y-%m-%d %H:%M:00')
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])
dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx
0,2.280913,2022-08-07 01:00:00,39,1169764903,688520770,887,1.663079,6582
1,15.432999,2022-08-07 02:00:00,237,7110027747,3880719039,7752,11.868786,45632
2,24.098700,2022-08-07 03:00:00,236,7080964616,3738681213,8183,15.676680,44685
3,20.704683,2022-08-07 04:00:00,259,7770467023,4057887078,10664,27.055056,51861
4,14.718397,2022-08-07 05:00:00,260,7799439368,4218450045,10480,11.841531,51113
...,...,...,...,...,...,...,...,...
601,72.111645,2022-09-01 02:00:00,262,7860700504,4377071552,6209,17.266260,45688
602,61.364488,2022-09-01 03:00:00,245,7350614227,3984931896,8060,17.113234,48005
603,45.194583,2022-09-01 04:00:00,241,7230584402,3824712180,8929,14.133103,40814
604,39.157729,2022-09-01 05:00:00,261,7831988695,4187495091,7087,15.891177,40512


In [17]:
dataset_tip = pd.read_csv('./datasets/dataset_validation_threshold.csv')
dataset_tip = dataset_tip.drop(columns=['Unnamed: 0'])
dataset_tip = dataset_tip.rename(columns={"block_time":"timestamp"})
dataset_tip['timestamp'] = pd.to_datetime(dataset_tip['timestamp'])
dataset_tip['timestamp'] = dataset_tip['timestamp'].dt.strftime('%Y-%m-%d %H:%M:00')
dataset_tip['timestamp'] = pd.to_datetime(dataset_tip['timestamp'])

dataset = dataset.merge(dataset_tip, on='timestamp', how='left', indicator=True)
dataset = dataset.drop(columns=['_merge'])

dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,q3_tip
0,2.280913,2022-08-07 01:00:00,39,1169764903,688520770,887,1.663079,6582,0.529009,0.168829,0.715017
1,15.432999,2022-08-07 02:00:00,237,7110027747,3880719039,7752,11.868786,45632,0.259681,0.143453,0.518556
2,24.098700,2022-08-07 03:00:00,236,7080964616,3738681213,8183,15.676680,44685,0.213495,0.114983,0.458896
3,20.704683,2022-08-07 04:00:00,259,7770467023,4057887078,10664,27.055056,51861,0.194821,0.118158,0.476867
4,14.718397,2022-08-07 05:00:00,260,7799439368,4218450045,10480,11.841531,51113,0.168015,0.102764,0.338216
...,...,...,...,...,...,...,...,...,...,...,...
601,72.111645,2022-09-01 02:00:00,262,7860700504,4377071552,6209,17.266260,45688,0.260117,0.149365,0.456055
602,61.364488,2022-09-01 03:00:00,245,7350614227,3984931896,8060,17.113234,48005,0.204441,0.128885,0.381101
603,45.194583,2022-09-01 04:00:00,241,7230584402,3824712180,8929,14.133103,40814,0.222156,0.109491,0.345284
604,39.157729,2022-09-01 05:00:00,261,7831988695,4187495091,7087,15.891177,40512,0.241508,0.128553,0.419928


In [18]:
eth_price = pd.read_csv('./datasets/eth_price_validation.csv')
eth_price['datetime'] = pd.to_datetime(eth_price['datetime'])
eth_price = eth_price[eth_price['datetime'] <= '2022-09-21 09:35:00']
eth_price = eth_price.rename(columns={'datetime': 'timestamp'})
eth_price = eth_price.drop(columns=['Unnamed: 0'])
eth_price = eth_price.drop_duplicates(subset=['timestamp'])
eth_price

,timestamp,eth_price
0,2022-08-07 01:00:00,1677.00
1,2022-08-07 02:00:00,1685.98
2,2022-08-07 03:00:00,1684.07
3,2022-08-07 04:00:00,1675.00
4,2022-08-07 05:00:00,1681.39
...,...,...
995,2022-09-17 12:00:00,1424.67
996,2022-09-17 13:00:00,1422.80
997,2022-09-17 14:00:00,1445.32
998,2022-09-17 15:00:00,1444.34


In [19]:
dataset = dataset.merge(eth_price, on='timestamp', how='left', indicator=True)

dataset = dataset.drop(columns=['_merge'])
dataset['date'] = pd.to_datetime(dataset['timestamp'].dt.date) - datetime.timedelta(days=1)
dataset = dataset.dropna()
dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,q3_tip,eth_price,date
0,2.280913,2022-08-07 01:00:00,39,1169764903,688520770,887,1.663079,6582,0.529009,0.168829,0.715017,1677.00,2022-08-06
1,15.432999,2022-08-07 02:00:00,237,7110027747,3880719039,7752,11.868786,45632,0.259681,0.143453,0.518556,1685.98,2022-08-06
2,24.098700,2022-08-07 03:00:00,236,7080964616,3738681213,8183,15.676680,44685,0.213495,0.114983,0.458896,1684.07,2022-08-06
3,20.704683,2022-08-07 04:00:00,259,7770467023,4057887078,10664,27.055056,51861,0.194821,0.118158,0.476867,1675.00,2022-08-06
4,14.718397,2022-08-07 05:00:00,260,7799439368,4218450045,10480,11.841531,51113,0.168015,0.102764,0.338216,1681.39,2022-08-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,72.111645,2022-09-01 02:00:00,262,7860700504,4377071552,6209,17.266260,45688,0.260117,0.149365,0.456055,1557.13,2022-08-31
602,61.364488,2022-09-01 03:00:00,245,7350614227,3984931896,8060,17.113234,48005,0.204441,0.128885,0.381101,1549.91,2022-08-31
603,45.194583,2022-09-01 04:00:00,241,7230584402,3824712180,8929,14.133103,40814,0.222156,0.109491,0.345284,1551.79,2022-08-31
604,39.157729,2022-09-01 05:00:00,261,7831988695,4187495091,7087,15.891177,40512,0.241508,0.128553,0.419928,1553.19,2022-08-31


In [20]:
# Fear and Green Index
fag = pd.read_csv('./datasets/fear_and_greed_index.csv')
fag['date'] = pd.to_datetime(fag['date'], format='%d-%m-%Y')

dataset = dataset.merge(fag, on='date', how='left', indicator=True)
dataset = dataset.drop(columns=['_merge'])
dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,q3_tip,eth_price,date,fng_value,fng_classification
0,2.280913,2022-08-07 01:00:00,39,1169764903,688520770,887,1.663079,6582,0.529009,0.168829,0.715017,1677.00,2022-08-06,31,Fear
1,15.432999,2022-08-07 02:00:00,237,7110027747,3880719039,7752,11.868786,45632,0.259681,0.143453,0.518556,1685.98,2022-08-06,31,Fear
2,24.098700,2022-08-07 03:00:00,236,7080964616,3738681213,8183,15.676680,44685,0.213495,0.114983,0.458896,1684.07,2022-08-06,31,Fear
3,20.704683,2022-08-07 04:00:00,259,7770467023,4057887078,10664,27.055056,51861,0.194821,0.118158,0.476867,1675.00,2022-08-06,31,Fear
4,14.718397,2022-08-07 05:00:00,260,7799439368,4218450045,10480,11.841531,51113,0.168015,0.102764,0.338216,1681.39,2022-08-06,31,Fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,72.111645,2022-09-01 02:00:00,262,7860700504,4377071552,6209,17.266260,45688,0.260117,0.149365,0.456055,1557.13,2022-08-31,23,Extreme Fear
602,61.364488,2022-09-01 03:00:00,245,7350614227,3984931896,8060,17.113234,48005,0.204441,0.128885,0.381101,1549.91,2022-08-31,23,Extreme Fear
603,45.194583,2022-09-01 04:00:00,241,7230584402,3824712180,8929,14.133103,40814,0.222156,0.109491,0.345284,1551.79,2022-08-31,23,Extreme Fear
604,39.157729,2022-09-01 05:00:00,261,7831988695,4187495091,7087,15.891177,40512,0.241508,0.128553,0.419928,1553.19,2022-08-31,23,Extreme Fear


In [21]:
# TVL dataset from Defillama
tvl = pd.read_csv('./datasets/defillama_tvl.csv')
tvl['date'] = pd.to_datetime(tvl['date'], format='%d/%m/%Y')
tvl = tvl.drop(columns=['Unnamed: 0'])

dataset = dataset.merge(tvl, on='date', how='left', indicator=True)
dataset = dataset.drop(columns=['_merge'])
dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,q3_tip,eth_price,date,fng_value,fng_classification,ethereum_tvl,polygon_tvl,bsc_tvl
0,2.280913,2022-08-07 01:00:00,39,1169764903,688520770,887,1.663079,6582,0.529009,0.168829,0.715017,1677.00,2022-08-06,31,Fear,3.730499e+10,1.611113e+09,5.795399e+09
1,15.432999,2022-08-07 02:00:00,237,7110027747,3880719039,7752,11.868786,45632,0.259681,0.143453,0.518556,1685.98,2022-08-06,31,Fear,3.730499e+10,1.611113e+09,5.795399e+09
2,24.098700,2022-08-07 03:00:00,236,7080964616,3738681213,8183,15.676680,44685,0.213495,0.114983,0.458896,1684.07,2022-08-06,31,Fear,3.730499e+10,1.611113e+09,5.795399e+09
3,20.704683,2022-08-07 04:00:00,259,7770467023,4057887078,10664,27.055056,51861,0.194821,0.118158,0.476867,1675.00,2022-08-06,31,Fear,3.730499e+10,1.611113e+09,5.795399e+09
4,14.718397,2022-08-07 05:00:00,260,7799439368,4218450045,10480,11.841531,51113,0.168015,0.102764,0.338216,1681.39,2022-08-06,31,Fear,3.730499e+10,1.611113e+09,5.795399e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,72.111645,2022-09-01 02:00:00,262,7860700504,4377071552,6209,17.266260,45688,0.260117,0.149365,0.456055,1557.13,2022-08-31,23,Extreme Fear,3.367994e+10,1.747198e+09,5.341860e+09
602,61.364488,2022-09-01 03:00:00,245,7350614227,3984931896,8060,17.113234,48005,0.204441,0.128885,0.381101,1549.91,2022-08-31,23,Extreme Fear,3.367994e+10,1.747198e+09,5.341860e+09
603,45.194583,2022-09-01 04:00:00,241,7230584402,3824712180,8929,14.133103,40814,0.222156,0.109491,0.345284,1551.79,2022-08-31,23,Extreme Fear,3.367994e+10,1.747198e+09,5.341860e+09
604,39.157729,2022-09-01 05:00:00,261,7831988695,4187495091,7087,15.891177,40512,0.241508,0.128553,0.419928,1553.19,2022-08-31,23,Extreme Fear,3.367994e+10,1.747198e+09,5.341860e+09


In [22]:
# Polygon dataset
polygon = pd.read_csv('./datasets/polygon_validation.csv')
polygon = polygon.rename(columns={"block_time":"timestamp", "median_tip":"polygon_median_tip", "q1_tip":"polygon_q1_tip", "q3_tip":"polygon_q3_tip", "base_fee":"polygon_base_fee", "blocks":"polygon_blocks"
                                    , "gas_limit":"polygon_gas_limit", "gas_used":"polygon_gas_used", "legacy_tx":"polygon_legacy_tx", "sum_priority_fee_eth":"polygon_sum_priority_fee_eth", "tx":"polygon_tx"})
polygon['timestamp'] = pd.to_datetime(polygon['timestamp'])
polygon['timestamp'] = polygon['timestamp'].dt.strftime('%Y-%m-%d %H:%M:00')
polygon['timestamp'] = pd.to_datetime(polygon['timestamp'])
polygon = polygon[polygon['timestamp'] <= '2022-09-21 09:35:00']
polygon = polygon[polygon['timestamp'] >= '2021-08-05 12:00:00']
polygon = polygon.drop(columns=['Unnamed: 0'])
polygon = polygon.drop_duplicates(subset=['timestamp'])
polygon = polygon.fillna(0)

# Join dataset and eth price
dataset = dataset.merge(polygon, on='timestamp', how='left', indicator=True)
for c in [s for s in dataset.columns if s in ['polygon_median_tip', 'polygon_q1_tip', 'polygon_q3_tip', 'polygon_base_fee', 'polygon_blocks', 'polygon_gas_limit', 'polygon_gas_used', 'polygon_legacy_tx', 'polygon_legacy_tx', 'polygon_sum_priority_fee_eth', 'polygon_tx']]:
    dataset[c].fillna(0, inplace=True)

dataset = dataset.drop(columns=['_merge'])

dataset


,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,...,polygon_base_fee,polygon_blocks,polygon_gas_limit,polygon_gas_used,polygon_legacy_tx,polygon_sum_priority_fee_eth,polygon_tx,polygon_median_tip,polygon_q1_tip,polygon_q3_tip
0,2.280913,2022-08-07 01:00:00,39,1169764903,688520770,887,1.663079,6582,0.529009,0.168829,...,1.254390e-05,122,3517426897,2436365166,7690,146.847203,13014,30.000000,30.0,30.005361
1,15.432999,2022-08-07 02:00:00,237,7110027747,3880719039,7752,11.868786,45632,0.259681,0.143453,...,9.997895e-06,1427,41472938081,16512644188,53273,810.512477,92663,30.000000,30.0,30.000000
2,24.098700,2022-08-07 03:00:00,236,7080964616,3738681213,8183,15.676680,44685,0.213495,0.114983,...,5.527889e-05,1458,42972534147,16943450885,54394,1032.425449,89917,30.000000,30.0,30.000000
3,20.704683,2022-08-07 04:00:00,259,7770467023,4057887078,10664,27.055056,51861,0.194821,0.118158,...,1.841318e-05,1458,42918580993,18549582184,54654,1192.101887,86380,30.000000,30.0,30.000000
4,14.718397,2022-08-07 05:00:00,260,7799439368,4218450045,10480,11.841531,51113,0.168015,0.102764,...,7.373709e-06,1478,43776459114,18573703238,68009,1146.085489,101144,30.000000,30.0,30.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,72.111645,2022-09-01 02:00:00,262,7860700504,4377071552,6209,17.266260,45688,0.260117,0.149365,...,2.294580e-06,1656,48299217188,21383904896,115366,2659.286491,177445,30.242697,30.0,30.693160
602,61.364488,2022-09-01 03:00:00,245,7350614227,3984931896,8060,17.113234,48005,0.204441,0.128885,...,9.519104e-06,1725,50461462031,20423586691,135958,1972.937626,180160,30.000000,30.0,30.242695
603,45.194583,2022-09-01 04:00:00,241,7230584402,3824712180,8929,14.133103,40814,0.222156,0.109491,...,3.182969e-07,1654,48019809715,17413540713,58986,1576.754359,101163,30.000000,30.0,30.108800
604,39.157729,2022-09-01 05:00:00,261,7831988695,4187495091,7087,15.891177,40512,0.241508,0.128553,...,2.224475e-07,1731,51069007883,16628431109,52942,1362.597335,95858,30.000000,30.0,30.000000


In [23]:
# BSC dataset
bsc = pd.read_csv('./datasets/bsc_validation.csv')
bsc = bsc.rename(columns={"block_time":"timestamp","blocks":"bsc_blocks", "gas_limit":"bsc_gas_limit", "gas_used":"bsc_gas_used", "tx":"bsc_tx", 'hourly_median_fee_tx': 'bsc_hourly_median_fee_tx'})
bsc['timestamp'] = pd.to_datetime(bsc['timestamp'])
bsc['timestamp'] = bsc['timestamp'].dt.strftime('%Y-%m-%d %H:%M:00')
bsc['timestamp'] = pd.to_datetime(bsc['timestamp'])
bsc = bsc[bsc['timestamp'] <= '2022-09-21 09:35:00']
bsc = bsc[bsc['timestamp'] >= '2021-08-05 12:00:00']
bsc = bsc.drop(columns=['Unnamed: 0', 'legacy_tx'])
bsc = bsc.drop_duplicates(subset=['timestamp'])

# Join dataset and eth price
dataset = dataset.merge(bsc, on='timestamp', how='left', indicator=True)
dataset = dataset.drop(columns=['_merge'])
dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,...,polygon_sum_priority_fee_eth,polygon_tx,polygon_median_tip,polygon_q1_tip,polygon_q3_tip,bsc_blocks,bsc_gas_limit,bsc_gas_used,bsc_hourly_median_fee_tx,bsc_tx
0,2.280913,2022-08-07 01:00:00,39,1169764903,688520770,887,1.663079,6582,0.529009,0.168829,...,146.847203,13014,30.000000,30.0,30.005361,186,14937496983,2112628682,4.613248e+14,16204
1,15.432999,2022-08-07 02:00:00,237,7110027747,3880719039,7752,11.868786,45632,0.259681,0.143453,...,810.512477,92663,30.000000,30.0,30.000000,1200,96404272172,15007653702,3.791880e+14,121816
2,24.098700,2022-08-07 03:00:00,236,7080964616,3738681213,8183,15.676680,44685,0.213495,0.114983,...,1032.425449,89917,30.000000,30.0,30.000000,1200,96412928123,15311473031,3.973310e+14,119293
3,20.704683,2022-08-07 04:00:00,259,7770467023,4057887078,10664,27.055056,51861,0.194821,0.118158,...,1192.101887,86380,30.000000,30.0,30.000000,1200,96384534667,14615075224,3.731388e+14,118881
4,14.718397,2022-08-07 05:00:00,260,7799439368,4218450045,10480,11.841531,51113,0.168015,0.102764,...,1146.085489,101144,30.000000,30.0,30.000002,1200,96396605799,14773495447,3.685736e+14,121680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,72.111645,2022-09-01 02:00:00,262,7860700504,4377071552,6209,17.266260,45688,0.260117,0.149365,...,2659.286491,177445,30.242697,30.0,30.693160,1178,88330348515,16855648063,3.613400e+14,135111
602,61.364488,2022-09-01 03:00:00,245,7350614227,3984931896,8060,17.113234,48005,0.204441,0.128885,...,1972.937626,180160,30.000000,30.0,30.242695,1159,87002579227,16106844474,3.315525e+14,133969
603,45.194583,2022-09-01 04:00:00,241,7230584402,3824712180,8929,14.133103,40814,0.222156,0.109491,...,1576.754359,101163,30.000000,30.0,30.108800,1200,90079623633,17042095093,3.091848e+14,144558
604,39.157729,2022-09-01 05:00:00,261,7831988695,4187495091,7087,15.891177,40512,0.241508,0.128553,...,1362.597335,95858,30.000000,30.0,30.000000,1174,88138103727,16255971907,2.901000e+14,139423


In [24]:
# NFT Data
nft = pd.read_csv('./datasets/nft.csv')
nft = nft.rename(columns={"timestamp":"date"})
nft = nft.drop(columns=['Unnamed: 0'])
nft['date'] = pd.to_datetime(nft['date'], format='%Y-%m-%d')


dataset = dataset.merge(nft, on='date', how='left', indicator=True)
dataset = dataset.drop(columns=['_merge'])
dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,...,mfers_sales_count,deadfellaz_floor_usd,deadfellaz_volume_usd,deadfellaz_sales_count,beeple_floor_usd,beeple_volume_usd,beeple_sales_count,clonex_floor_usd,clonex_volume_usd,clonex_sales_count
0,2.280913,2022-08-07 01:00:00,39,1169764903,688520770,887,1.663079,6582,0.529009,0.168829,...,24,1376.048000,15485.486578,10,28735.120000,0.0,0,13436.704000,104043.798720,7
1,15.432999,2022-08-07 02:00:00,237,7110027747,3880719039,7752,11.868786,45632,0.259681,0.143453,...,24,1376.048000,15485.486578,10,28735.120000,0.0,0,13436.704000,104043.798720,7
2,24.098700,2022-08-07 03:00:00,236,7080964616,3738681213,8183,15.676680,44685,0.213495,0.114983,...,24,1376.048000,15485.486578,10,28735.120000,0.0,0,13436.704000,104043.798720,7
3,20.704683,2022-08-07 04:00:00,259,7770467023,4057887078,10664,27.055056,51861,0.194821,0.118158,...,24,1376.048000,15485.486578,10,28735.120000,0.0,0,13436.704000,104043.798720,7
4,14.718397,2022-08-07 05:00:00,260,7799439368,4218450045,10480,11.841531,51113,0.168015,0.102764,...,24,1376.048000,15485.486578,10,28735.120000,0.0,0,13436.704000,104043.798720,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,72.111645,2022-09-01 02:00:00,262,7860700504,4377071552,6209,17.266260,45688,0.260117,0.149365,...,8,1040.413718,14744.637461,12,27915.435336,0.0,0,10886.242192,569498.519548,38
602,61.364488,2022-09-01 03:00:00,245,7350614227,3984931896,8060,17.113234,48005,0.204441,0.128885,...,8,1040.413718,14744.637461,12,27915.435336,0.0,0,10886.242192,569498.519548,38
603,45.194583,2022-09-01 04:00:00,241,7230584402,3824712180,8929,14.133103,40814,0.222156,0.109491,...,8,1040.413718,14744.637461,12,27915.435336,0.0,0,10886.242192,569498.519548,38
604,39.157729,2022-09-01 05:00:00,261,7831988695,4187495091,7087,15.891177,40512,0.241508,0.128553,...,8,1040.413718,14744.637461,12,27915.435336,0.0,0,10886.242192,569498.519548,38


In [25]:
dataset.to_csv('./datasets/dataset_validation.csv')